In [ ]:
import pandas as pd
from meetup import get_meetup_events
from mhub import get_mhub_events
from luma import get_luma_group_events
from common import upload_to_gsheets, get_gsheet_df, create_event_markdown, create_discord_list
from datetime import datetime

# TODO: Get 1871 Events
# TODO: Get lu.ma users Events
# TODO: Fix luma
# TODO: Get Eventbrite Events
# TODO: Figure out the list of others that need to be scraped
# TODO: Figure out a way to make this format nicely for pasting into slack/discord
# TODO: Make a website/calendar
# TODO: Filter non-Chicago events from meetup groups (e.g. https://www.meetup.com/pydatachi/events/303000483/)
# TODO: Split slack messages into multiple markdowns to make life easier
# TODO: Throw this into ChatGPT to simplify the list?
# TODO: Add Organization to Markdown

In [ ]:
organization_df = get_gsheet_df("Organizations")

In [ ]:
meetup_groups = organization_df['Meetup'].tolist()
meetup_groups = list(set([x for x in meetup_groups if len(x) > 0]))

luma_groups = organization_df['LuMa'].tolist()
luma_groups = list(set([x for x in luma_groups if len(x) > 0]))

In [ ]:
organization_df

In [ ]:
try:
    meetup_df = get_meetup_events(meetup_groups, sleep_time=2, verbose=True, debug=True)
except Exception as e:
    print('Failed to get Meetup events')
    print(e)
try:
    mhub_df = get_mhub_events()
except Exception as e:
    print('Failed to get MHub events')
    print(e)
try:
    luma_df = get_luma_group_events(luma_groups, sleep_time=1, verbose=False, debug=True)
except Exception as e:
    print('Failed to get LuMa events')
    print(e)

In [ ]:
# combine the dataframes
try:
    combined_df = pd.concat([meetup_df, mhub_df, luma_df], ignore_index=True)
except Exception as e:
    print(e)
    combined_df = pd.concat([mhub_df, meetup_df], ignore_index=True)

combined_df

In [ ]:
upload_to_gsheets(combined_df, "Events",['eventURL'], verbose=True)

In [ ]:
events_df = get_gsheet_df("Events")
create_event_markdown(events_df, "Chicago Events")

events_df = get_gsheet_df("Events")
create_discord_list(events_df, "Chicago Events")